# 🌤️ Real-time AQI + Weather Data Fetcher

In [ ]:
from google.colab import userdata
import requests
import pandas as pd
from datetime import datetime, timezone

# --- API Key ---
API_KEY = userdata.get('OPENWEATHER_API_KEY')
print("✅ API key loaded successfully.")

# --- Coordinates for Karachi ---
LAT, LON = 24.8607, 67.0011

def get_realtime_data():
    try:
        # --- Fetch Weather Data ---
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units=metric"
        weather = requests.get(weather_url, timeout=10).json()

        # --- Fetch Air Pollution Data ---
        pollution_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={LAT}&lon={LON}&appid={API_KEY}"
        pollution = requests.get(pollution_url, timeout=10).json()

        # --- Validate Response ---
        if "main" not in weather or "list" not in pollution:
            raise ValueError("Invalid API response format")

        # --- Extract Weather Data ---
        temp = weather["main"]["temp"]
        humidity = weather["main"]["humidity"]
        wind_speed = weather["wind"]["speed"]

        # --- Extract Air Quality Data ---
        air = pollution["list"][0]
        aqi = air["main"]["aqi"]
        comp = air["components"]

        # --- Combine into Single Dictionary ---
        now = datetime.now(timezone.utc)
        data = {
            "datetime": now,
            "datetime_str": now.strftime("%Y-%m-%d %H:%M:%S"),
            "city": "Karachi",
            "temp": temp,
            "humidity": humidity,
            "wind_speed": wind_speed,
            "aqi": aqi,
            "pm2_5": comp.get("pm2_5"),
            "pm10": comp.get("pm10"),
            "co": comp.get("co"),
            "no2": comp.get("no2"),
            "so2": comp.get("so2"),
            "o3": comp.get("o3"),
        }

        return data

    except Exception as e:
        print(f"⚠️ Error fetching data: {e}")
        return None


# --- Fetch and Save Data ---
data = get_realtime_data()
if data:
    df = pd.DataFrame([data])
    filename = "realtime_aqi_weather.csv"
    df.to_csv(filename, index=False)
    print(f"✅ Real-time data saved to {filename}")
    display(df)
else:
    print("❌ No data fetched.")
